In [2]:
# Módulos
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [3]:
# Dados
Tarefas = [i for i in range(6)]
Maquinas = [1, 2]
proc_tempo = {0: 0, 1: 64, 2: 53, 3: 63, 4: 99, 5: 189}
entrega = {0: 0, 1: 100, 2: 70, 3: 150, 4: 601, 5: 118}
M = 5000

In [4]:
# Modelo
model = ConcreteModel()

# Conjuntos
model.T = Set(initialize=Tarefas)
model.M = Set(initialize=Maquinas)
T, M = model.T, model.M

# Parâmetros
model.p = Param(T, initialize=proc_tempo, within=NonNegativeReals)
model.d = Param(T, initialize=entrega, within=NonNegativeReals)
p, d = model.p, model.d

# Variáveis de decisão
model.x = Var(T * M, within=Binary)
model.C = Var(M, within=NonNegativeReals)
model.C_max = Var(within=NonNegativeReals)
x, C, C_max = model.x, model.C, model.C_max

# Função objetivo
model.obj = Objective(sense=minimize,
                      expr = C_max)

# Restrições
# Definição do instante final de processamento
model.r_proc_final = ConstraintList()
for k in M:
    model.r_proc_final.add(expr = C[k] == sum(p[i]*x[i,k] for i in T))
# Definição do makespan
model.r_mksp = ConstraintList()
for k in M:
    model.r_mksp.add(expr = C_max >= C[k])
# Todas as tarefas são processadas
model.r_proc = ConstraintList()
for i in T:
    model.r_proc.add(expr = sum(x[i,k] for k in M) == 1)

# model.pprint()

In [5]:
# Resolução
solver = SolverFactory('glpk')
resultado = solver.solve(model, tee=False)

In [6]:
# Apresentação de resultados
if (resultado.solver.status == SolverStatus.ok) and (resultado.solver.termination_condition != TerminationCondition.infeasible):
    for i in T:
        for k in M:
            if value(x[i,k]) == 1 and i != 0:
                print('A tarefa {} é processada na máquina {}'.format(i,k))
    print()
    print('O makespan mínimo é de {:.0f} minutos.'.format(value(model.obj)))
else:
    print(resultado.solver.termination_condition)

A tarefa 1 é processada na máquina 1
A tarefa 2 é processada na máquina 2
A tarefa 3 é processada na máquina 1
A tarefa 4 é processada na máquina 1
A tarefa 5 é processada na máquina 2

O makespan mínimo é de 242 minutos.
